In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
from library.pmf_estimators import *
from library.information_discrete import MI, E

In [2]:
df = sns.load_dataset("iris")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
df_discr = df.copy()

# Discretizing the Dataset
df_discr.iloc[:, :-1] = (df_discr.iloc[:, :-1] * 10).astype(int)

In [4]:
# defining some constants to use in our purpose
features = df.columns.to_list()[:-1]
rows, n_features = df_discr.iloc[:, :-1].shape

# Computing the Entropies for all the Iris features
Entropies = []
for feature in features:
    x_vals, pmf = pmf_univariate(df_discr[feature])  # Computing the pmf
    Entropies.append(E(pmf))

for i in range(n_features):
    print(f'{features[i]}: {Entropies[i]}\n')

sepal_length: 3.3423682425704624

sepal_width: 2.7886565850017027

petal_length: 3.4896979687210656

petal_width: 2.807126229460098



In [5]:
MI_matrix = np.zeros((n_features, n_features))

# Computing the Mutual information over all the features combinations
for i in combinations(range(n_features), 2):
    ix, iy = i                          # extracting the indices 
    Lx, Ly = features[ix], features[iy] # extracting the features
    PXY = Joint_p(df_discr[Lx], df_discr[Ly]) # Computing the Joint Probability
    PX = PXY.sum(axis=1)  # Computing the X marginal probability
    PY = PXY.sum(axis=0)  # Computing the Y marginal probability
    MI_matrix[ix, iy] = MI_matrix[iy, ix] = MI(PXY, PX, PY)  # Mutual Information is symmetric


# Converting the result into a pandas dataframe
MI_matrix = pd.DataFrame(MI_matrix, columns=features, index=features)
MI_matrix.columns.name = 'Mutual Information'
MI_matrix[MI_matrix == 0] = Entropies # adding the entropies to the result
MI_matrix

Mutual Information,sepal_length,sepal_width,petal_length,petal_width
sepal_length,3.342368,1.448570,2.081429,1.553124
sepal_width,1.448570,2.788657,1.543936,1.161699
petal_length,2.081429,1.543936,3.489698,1.867926
petal_width,1.553124,1.161699,1.867926,2.807126
